<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/NLP_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**This file processes the text data with NLP algorithms**

In [ ]:
#Tokenize the Words
from nltk.tokenize import RegexpTokenizer

# Create tokenizer
tokenizer = RegexpTokenizer('\w+')

tokens = tokenizer.tokenize(pagetext)
tokens[:8]

In [ ]:
#Change to lower case 
words = []


# Loop through list tokens and make lower case
for word in tokens:
    words.append(word.lower())


# Print several items from list as sanity check
words[:8]

In [ ]:
# Import nltk
import nltk

nltk.download('stopwords')

# Get English stopwords and print some of them
sw = nltk.corpus.stopwords.words('german')
sw[:5]

In [ ]:
#Remove German stopwords
# Initialize new list
words_ns = []

# Add to words_ns all words that are in words but not in sw
for word in words:
    if word not in sw:
        words_ns.append(word)

# Print several list items as sanity check
words_ns[:5]

**Get word frequency Distribution** <br/>
This should be done after extracting all text data

In [ ]:
#Get Word Frequency Distribution
import matplotlib.pyplot as plt
import seaborn as sns

#Display Inline chart
%matplotlib inline 
sns.set()

#Create freq distribution
freqdist1 =  nltk.FreqDist(words_ns)
freqdist1.plot(25)

In [ ]:
#Chunking of Data

ne_chunks = nltk.batch_ne_chunk(words_ns)

In [ ]:
Install Additional stop words - stop-words - If Required. Curren

https://pypi.org/project/stop-words/


In [ ]:
#Import statements for processing
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem.snowball import SnowballStemmer        # module for stemming
#from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

In [ ]:
#Supports multipls languages- This stemmer is useful for search
print(" ".join(SnowballStemmer.languages))

arabic danish dutch english finnish french german hungarian italian norwegian porter portuguese romanian russian spanish swedish


In [ ]:
#Create a stemmer ignoring stop words
stemmer2 = SnowballStemmer("german", ignore_stopwords=True)
